##1) Demanda por zona y mes: ¿cuáles son las 10 zonas con más viajes por mes? (PU y DO por separado).

In [ ]:
WITH MonthlyTrips AS (
    SELECT
        dd.YEAR_MONTH_KEY,
        -- Utilizamos YEAR_MONTH_KEY para ordenar por mes y año
        pz.BOROUGH AS PICKUP_BOROUGH, -- Atributo de DIM_ZONES
        pz.ZONE_ID AS PICKUP_ZONE,    -- Atributo de DIM_ZONES
        dz.BOROUGH AS DROPOFF_BOROUGH,
        dz.ZONE_ID AS DROPOFF_ZONE,
        COUNT(*) AS TOTAL_TRIPS
    FROM NY_TAXI2.RAW.FCT_TRIPS ft
    JOIN NY_TAXI2.RAW.DIM_DATE dd ON ft.PICKUP_DATE_SK = dd.DATE_SK
    JOIN NY_TAXI2.RAW.DIM_ZONES pz ON ft.PU_ZONE_SK = pz.ZONE_SK  -- Zona de recogida
    JOIN NY_TAXI2.RAW.DIM_ZONES dz ON ft.DO_ZONE_SK = dz.ZONE_SK  -- Zona de entrega
    GROUP BY 1, 2,3,4,5
),
RankedPU AS (
    SELECT
        *,
        RANK() OVER (PARTITION BY YEAR_MONTH_KEY ORDER BY TOTAL_TRIPS DESC) as pu_rank
    FROM MonthlyTrips
),
RankedDO AS (
    SELECT
        YEAR_MONTH_KEY,
        DROPOFF_ZONE,
        TOTAL_TRIPS,
        RANK() OVER (PARTITION BY YEAR_MONTH_KEY ORDER BY TOTAL_TRIPS DESC) as do_rank
    FROM MonthlyTrips
)

-- Resultado final: Unión de Top 10 PU y Top 10 DO
SELECT 'PICKUP' AS TYPE, YEAR_MONTH_KEY, PICKUP_ZONE AS ZONE, TOTAL_TRIPS, pu_rank AS RANK
FROM RankedPU WHERE pu_rank <= 10

UNION ALL

SELECT 'DROPOFF' AS TYPE, YEAR_MONTH_KEY, DROPOFF_ZONE AS ZONE, TOTAL_TRIPS, do_rank AS RANK
FROM RankedDO WHERE do_rank <= 10

ORDER BY YEAR_MONTH_KEY, TYPE, RANK;

,TYPE,YEAR_MONTH_KEY,ZONE,TOTAL_TRIPS,RANK
0,DROPOFF,2015-01,NaN,214411,1
1,DROPOFF,2015-01,Upper East Side North,59540,2
2,DROPOFF,2015-01,Upper East Side North,54032,3
3,DROPOFF,2015-01,Upper East Side South,51577,4
4,DROPOFF,2015-01,Upper East Side South,46985,5
...,...,...,...,...,...
2555,PICKUP,2025-08,Upper East Side South,7906,6
2556,PICKUP,2025-08,JFK Airport,7454,7
2557,PICKUP,2025-08,JFK Airport,6795,8
2558,PICKUP,2025-08,Midtown Center,6445,9


##2) Ingresos y propinas: ¿cómo varían los ingresos totales y el tip % por borough y mes?

In [ ]:
SELECT
    dd.YEAR_MONTH_KEY,
    pz.BOROUGH, -- Atributo de DIM_ZONES
    SUM(ft.TOTAL_AMOUNT) AS TOTAL_REVENUE,
    SUM(ft.TIP_AMOUNT) AS TOTAL_TIPS,
    -- Calcula el porcentaje de propina: (Propinas / Tarifa Base) * 100
    (SUM(ft.TIP_AMOUNT) / NULLIF(SUM(ft.FARE_AMOUNT), 0)) * 100 AS TIP_PERCENTAGE
FROM NY_TAXI2.RAW.FCT_TRIPS ft
JOIN NY_TAXI2.RAW.DIM_DATE dd ON ft.PICKUP_DATE_SK = dd.DATE_SK
JOIN NY_TAXI2.RAW.DIM_ZONES pz ON ft.PU_ZONE_SK = pz.ZONE_SK -- Usamos la zona de recogida pa
GROUP BY 1, 2
ORDER BY YEAR_MONTH_KEY, BOROUGH;

,YEAR_MONTH_KEY,BOROUGH,TOTAL_REVENUE,TOTAL_TIPS,TIP_PERCENTAGE
0,2015-01,Bronx,1.566356e+05,8194.69,6.085519
1,2015-01,Brooklyn,3.912814e+06,418719.29,13.108521
2,2015-01,EWR,4.446723e+04,5518.18,14.345686
3,2015-01,Manhattan,1.589843e+08,20041419.25,16.069695
4,2015-01,Queens,2.506706e+07,2671631.92,13.201921
...,...,...,...,...,...
1019,2025-08,Manhattan,5.309222e+07,6736706.46,19.794540
1020,2025-08,Queens,2.201036e+07,2519914.93,15.382407
1021,2025-08,Staten Island,3.046050e+03,429.38,17.092472
1022,2025-08,Unknown,1.638027e+05,20318.72,17.758869


##3) Velocidad y congestión: promedio de mph por franja horaria y borough (viajes diurnos vs. nocturnos).

In [ ]:
WITH TripSpeed AS (
    SELECT
        pz.BOROUGH,
        ft.TRIP_DURATION_SECONDS,
        ft.TRIP_DISTANCE,
        dt.HOUR_OF_DAY, -- Atributo de DIM_TIME (Query 5)

        CASE
            WHEN dt.HOUR_OF_DAY >= 6 AND dt.HOUR_OF_DAY < 22 THEN 'DIURNO (6h-22h)' -- Mañana, Tarde, Noche
            ELSE 'NOCTURNO (22h-6h)' -- Madrugada
        END AS DAY_NIGHT_BAND,

        -- Cálculo de MPH:
        (ft.TRIP_DISTANCE / NULLIF(ft.TRIP_DURATION_SECONDS, 0)) * 3600 AS TRIP_MPH
    FROM NY_TAXI2.RAW.FCT_TRIPS ft
    JOIN NY_TAXI2.RAW.DIM_ZONES pz ON ft.PU_ZONE_SK = pz.ZONE_SK
    JOIN NY_TAXI2.RAW.DIM_TIME dt ON ft.PICKUP_TIME_SK = dt.TIME_SK
    -- Filtramos para evitar valores no válidos
    WHERE ft.TRIP_DURATION_SECONDS > 0 AND ft.TRIP_DISTANCE > 0
)
SELECT
    BOROUGH,
    DAY_NIGHT_BAND,
    HOUR_OF_DAY,
    AVG(TRIP_MPH) AS AVG_MPH
FROM TripSpeed
GROUP BY 1, 2, 3
ORDER BY BOROUGH, HOUR_OF_DAY;

,BOROUGH,DAY_NIGHT_BAND,HOUR_OF_DAY,AVG_MPH
0,Bronx,NOCTURNO (22h-6h),0,53.176832
1,Bronx,NOCTURNO (22h-6h),1,53.293208
2,Bronx,NOCTURNO (22h-6h),2,47.363421
3,Bronx,NOCTURNO (22h-6h),3,53.977338
4,Bronx,NOCTURNO (22h-6h),4,50.134790
...,...,...,...,...
187,NaN,DIURNO (6h-22h),19,511.094205
188,NaN,DIURNO (6h-22h),20,514.876568
189,NaN,DIURNO (6h-22h),21,499.784208
190,NaN,NOCTURNO (22h-6h),22,623.728190


##4) Duración del viaje: percentiles (p50/p90) de duración (en segundos) por PULocationID (pickup)

In [ ]:
SELECT
    ft.PU_ZONE_SK,
    pz.ZONE_ID,
    pz.BOROUGH,

    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY ft.TRIP_DURATION_SECONDS) AS P50_DURATION_SECONDS,
    PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY ft.TRIP_DURATION_SECONDS) AS P90_DURATION_SECONDS
FROM NY_TAXI2.RAW.FCT_TRIPS ft
JOIN NY_TAXI2.RAW.DIM_ZONES pz ON ft.PU_ZONE_SK = pz.ZONE_SK
WHERE ft.TRIP_DURATION_SECONDS > 0
GROUP BY 1, 2, 3
ORDER BY P90_DURATION_SECONDS DESC;

,PU_ZONE_SK,ZONE_ID,BOROUGH,P50_DURATION_SECONDS,P90_DURATION_SECONDS
0,5,Arden Heights,Staten Island,3503.5,5800.2
1,86,Far Rockaway,Queens,3320.0,5782.1
2,117,Hammels/Arverne,Queens,3091.0,5541.0
3,118,Heartland Village/Todt Hill,Staten Island,1058.0,5113.0
4,201,Rockaway Park,Queens,2423.5,5107.0
...,...,...,...,...,...
260,141,Lenox Hill West,Manhattan,537.0,1303.0
261,104,Governor's Island/Ellis Island/Liberty Island,Manhattan,576.0,1246.6
262,265,Outside of NYC,NaN,43.0,1162.0
263,103,Governor's Island/Ellis Island/Liberty Island,Manhattan,833.0,910.6


##5) Elasticidad temporal: distribución de viajes por día de semana y hora; ¿cuáles son las horas pico?

In [ ]:
SELECT
    dd.DAY_OF_WEEK_ISO,
    dd.DAY_OF_WEEK_NAME,
    dt.HOUR_OF_DAY,
    COUNT(*) AS TOTAL_TRIPS
FROM NY_TAXI2.RAW.FCT_TRIPS ft
JOIN NY_TAXI2.RAW.DIM_DATE dd ON ft.PICKUP_DATE_SK = dd.DATE_SK
JOIN NY_TAXI2.RAW.DIM_TIME dt ON ft.PICKUP_TIME_SK = dt.TIME_SK
GROUP BY 1, 2, 3
ORDER BY dd.DAY_OF_WEEK_ISO, dt.HOUR_OF_DAY;

SELECT
    dd.DAY_OF_WEEK_NAME,
    dt.HOUR_OF_DAY,
    COUNT(*) AS TOTAL_TRIPS
FROM NY_TAXI2.RAW.FCT_TRIPS ft
JOIN NY_TAXI2.RAW.DIM_DATE dd ON ft.PICKUP_DATE_SK = dd.DATE_SK
JOIN NY_TAXI2.RAW.DIM_TIME dt ON ft.PICKUP_TIME_SK = dt.TIME_SK
GROUP BY 1, 2
ORDER BY TOTAL_TRIPS DESC
LIMIT 5;

,DAY_OF_WEEK_ISO,DAY_OF_WEEK_NAME,HOUR_OF_DAY,TOTAL_TRIPS
0,1,Mon,0,1985672
1,1,Mon,1,1171009
2,1,Mon,2,746705
3,1,Mon,3,527406
4,1,Mon,4,589256
...,...,...,...,...
163,7,Sun,19,5171741
164,7,Sun,20,4683165
165,7,Sun,21,4432195
166,7,Sun,22,3894985
